Machine Learning with Pytorch

PyTorch is an open source framework for AI research and commercial production in machine learning. It is used to build, train and optmize deep learning neural networks for applications such as image recognition, natural language processing, and speech recognition. It provides computation support for CPU, GPU, parallel and distributed training on multiple GPUs and multiple nodes. PyTorch is also flexible and easily extensible, with specific libraries and tools available for many different domains. All of the above have made PyTorch a leading framework in machine learning.

This lab shows you how easy is to get started with PyTorch and use it to build, train and evaluate a neural network.

Objectives

After completing this lab you will be able to:

    - Install necessary PyTorch libraries;
    - Use PyTorch to build, train and evaluate neural networks.
    - Save the trained model parameters and use them later for inferencing.


Setup

For this lab, we will be using the following libraries:

    - torch
    - TorchVision   

Importing Required Libraries

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

How does this lab work?

The lab uses MNIST datasets. The dataset has over 60,000 images of hand written digits. The data will be partitioned between training the AI model and testing the AI model after training.

The main steps in this project include:

1. Download the MNIST dataset and create a DataLoader for the dataset.
2. Define an AI model to recognize a hand written digit.
3. Train the defined AI model using training data from the MNIST dataset.
4. Test the trained AI model using testing data from the MNIST dataset.
5. Evaluate

Download Dataset and Create Data Loader

The images are 28x28 pixel images of digits 0 through 9.

In [2]:
# Download training data from the MNIST datasets.
training_data = datasets.MNIST(
    root="..\Pytorch\Datasets",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="..\Pytorch\Datasets",
    train=False, 
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders to iterate over data
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("Training data size", len(train_dataloader) * batch_size)
print("Test data size:", len(test_dataloader) * batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:11: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:11: SyntaxWarning: invalid escape sequence '\P'
C:\Users\anthonny.paz\AppData\Local\Temp\ipykernel_11872\2042593826.py:3: SyntaxWarning: invalid escape sequence '\P'
  root="..\Pytorch\Datasets",
C:\Users\anthonny.paz\AppData\Local\Temp\ipykernel_11872\2042593826.py:11: SyntaxWarning: invalid escape sequence '\P'
  root="..\Pytorch\Datasets",


Training data size 60032
Test data size: 10048
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Define Model

We first determine the best device for perfoming training with cpu as the default device.

We then define the AI model as a neural network with 3 layers: an input layer, a hidden layer, and an output layer.
Between the layers, we use ReLU activation function.

Since the input images are 1x28x28 tensors, we need to flatten the input tensors into a 784 element tensor using the Flatten module before passing the input into our neural network.

In [3]:
# Get device for training.
device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() # Apple Silicon GPU
    else "cpu"
)
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, image_tensor):
        image_tensor = self.flatten(image_tensor)
        logits = self.linear_relu_stack(image_tensor)
        return logits
    
input_size = 28*28
hidden_size = 512
num_classes = 10

model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)


Using cpu device


Trainin Loop

We implement a training function to use with the train_dataloader to train our model. Each iteration over the dataloader returns a batch_size image data tensor along with the expected output. After moving the tensor to the device, we call the forward pass of our model, compute the predicition error using the expected output and then call the backwards pass to compute the gradients and apply them to the model parameters.

In [10]:
# Define our learning rate, loss function and optimizer
learning_rate = 1e-3 #0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Let's define our training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch_num, (X, y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        # Forward pass to compute prediction
        pred = model(X)
        # Compute prediction error using loss function
        loss = loss_fn(pred, y)

        # Backward pass
        optimizer.zero_grad() # zero any previous gradient calculations
        loss.backward() # calculate gradient
        optimizer.step() # update model parameters

        if batch_num > 0 and batch_num % 100 == 0:
            loss, current = loss.item(), batch_num * len(X)
            print(f"loss : {loss:>7f}   [{current:>5d}/{size:>5d}]")

Test Loop

The test methods evaluates the model's predictive perfomance using the test_dataloader. During testing, we don't require gradient computation, so we set the model in evaluate mode.

In [13]:
# Our test function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Train the Model

Now that we have defined methods to train our model and test the trained model's predictive behavior, lets train the model for 5 epochs over the dataset.

In [14]:
# Let's run training
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n-----------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------
loss : 0.024083   [ 6400/60000]
loss : 0.030919   [12800/60000]
loss : 0.080903   [19200/60000]
loss : 0.042963   [25600/60000]
loss : 0.119421   [32000/60000]
loss : 0.097621   [38400/60000]
loss : 0.085864   [44800/60000]
loss : 0.093243   [51200/60000]
loss : 0.064105   [57600/60000]
Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000109 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000164 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000143 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.002160 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000473 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.001503 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000460 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.002112 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000313 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.001286 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000828 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000489 

Test Error: 
 Accuracy: 0.6%, Avg loss: 0.000415 

Test Error: 
 Acc

Save the Model

Once we have trained a model, we can save the model parameters for future use in inferences. Here we save the state_dict of the model which contains the trained parameters. We then create a new instance of the model and load the previously save parameters into the new instance of the model. Finally we can inference using the new instance of the model.

In [23]:
# Save our model parameters
torch.save(model.state_dict, "ml_with_pytorch_model.pth")
print("Saved PyTorch Model State to Pytorch\models\ml_with_pytorch_model.pth")

# Load the saved model parameters into a new instance of the model
model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(torch.load("ml_with_pytorch_model.pth"))

# Inference using the new model instance
model.eval()
for i in range(10):
    x, y = test_data[i][0], test_data[i][1]

    x = x.to(device)
    pred = model(x)
    predicted, actual = pred[0].argmax(0).item(),y
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Saved PyTorch Model State to Pytorch\models\ml_with_pytorch_model.pth


<>:3: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\m'
C:\Users\anthonny.paz\AppData\Local\Temp\ipykernel_11872\1170887314.py:3: SyntaxWarning: invalid escape sequence '\m'
  print("Saved PyTorch Model State to Pytorch\models\ml_with_pytorch_model.pth")


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL getattr was not an allowed global by default. Please use `torch.serialization.add_safe_globals([getattr])` or the `torch.serialization.safe_globals([getattr])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pyt